# Customer Segmentation with RFM

## Business Problem(İş Problemi)

- Bir e-ticaret şirketi müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama stratejileri belirlemek istiyor.

- Kullanılacak "Online Retail Ⅱ"isimli veri seti İngiltere merkezli bir online satış mağazasının 01/12/2009 - 09/12/2011 tarihleri arasındaki satışlarını içermektedir.

- Şirket hediyelik eşyalar(promosyon ürünleri) satmaktadır.

- Şirketin müşterilerinin büyük bölümü toptancıdır yani kurumsal müşterilerdir.

In [1]:
## Veri Setini İnceleyelim

# Değişkenler
#####################################################################################################################
# InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. C ile başlıyorsa iptal edilen işlem.
# StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
# Description: Ürün ismi
# Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
# InvoiceDate: Fatura tarihi ve zamanı.
# UnitPrice: Ürün fiyatı (Sterlin cinsinden)
# CustomerID: Eşsiz müşteri numarası
# Country: Ülke ismi. Müşterinin yaşadığı ülke.
#######################################################################################################################

In [2]:
import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None) #yazdırma işlemi yaptığımızda bütün sütunların gözükmesini sağlar 
#pd.set_option('display.max_rows', None) #yazdırma işlemi yaptığımızda bütün satırların gözükmesini sağlar, çıktı çok kalabalık olmasın diye yorum satırına aldık 
pd.set_option('display.float_format', lambda x: '%.5f' % x)#sayısal değişkenlerin virgülden sonra kaç basamağını göstereceği bilgisi
# '%.5f' virgülden sonra 5 basamak gösterilecek demektir.

# bu excel dosyasında 2 sayfa var, bu analiz 2009-2010 yılları için yapılacağından sheet_name belirtilmelidir.
df_ = pd.read_excel("C:\\Users\\Monster\\Desktop\\dataa\\online_retail_II.xlsx", sheet_name="Year 2009-2010")
df = df_.copy()# kopyalama işlemi yapıyoruz, ilgili dataframe'in orijinalini bozmadan üzerinde çalışma imkanı sağlar. 
#eğer ilerde ters bir durum olursa ve veri setini tekrar okumamız gerekirse direkt olarak burayı çalıştıracağız yani veri setini tekrar okumaya gerek kalmayacak veri seti büyük olduğundan okuma işlemi biraz zaman alabilir.

In [3]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


- Veri setini inceleyecek olursak, buradaki "Invoice" değerleri(fatura Id'si) çoklama durumunda, bir faturada birden fazla ürün olabilir.
- Yani aynı Invoice değerine sahip olan ürünler aynı anda aynı kişi tarafından satın alınmış demektir.
- Veri setinde fatura Id'si, ürün kodu(StockCode), açıklaması, kaç adet satın alındığı, ne zaman alındığı, ilgili ürünün birim fiyatı, müşteri Id'si ve hangi ülkeden satın alındığı gibi bilgiler yer almaktadır.

In [4]:
df.shape

(525461, 8)

- 525461 gözlem birimi(row-satır) sayısı, 8 ise değişken(column-sütun) sayısıdır.

In [5]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [6]:
#eşsiz ürün sayısının ne olduğunu belirleyelim

df["Description"].nunique()

4681

In [7]:
# hangi üründen kaçar tane satıldığı bilgisini yazdıralım, bu kaç tane faturada bu ürünün yer aldığını döndürür

df["Description"].value_counts().head()

Description
WHITE HANGING HEART T-LIGHT HOLDER    3549
REGENCY CAKESTAND 3 TIER              2212
STRAWBERRY CERAMIC TRINKET BOX        1843
PACK OF 72 RETRO SPOT CAKE CASES      1466
ASSORTED COLOUR BIRD ORNAMENT         1457
Name: count, dtype: int64

In [8]:
# her bir üründen ne kadar sipariş verildiği bilgisi, bu ise toplamda kaç tane satıldığını döndürür
# yani ürün bir faturada yer alabilir ama o faturada kaç tane satılmış quantity'si nedir, bunu hesaplar


#description'a göre veri setini grupluyoruz, Quantity'lerin toplamını(sum) alıyoruz ve Quantity' göre azalan şekilde sıralıyoruz(sort_values ile)
df.groupby("Description").agg({"Quantity" : "sum"}).sort_values("Quantity", ascending=False).head()

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,57733
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54698
BROCADE RING PURSE,47647
PACK OF 72 RETRO SPOT CAKE CASES,46106
ASSORTED COLOUR BIRD ORNAMENT,44925


In [9]:
# eşsiz fatura sayısı

df["Invoice"].nunique()

28816

In [10]:
# faturanın toplam bedeli nedir

# Bu bir ürün için faturadaki toplam fiyatı belirler yani her satır için ayrı bir değer gelecek
df["TotalPrice"] = df["Price"] * df["Quantity"]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom,83.40000
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom,100.80000
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom,30.00000


In [11]:
# bir önceki işlemde her bir satır için yani her bir ürün için toplam fiyatı buldu, toplam fatura bedeli için aynı fatura Id'sine sahip olanları toplamamız gerekiyor.

# groupby("Invoice") Invoice değerine göre gruplar, yani Invoice'u aynı olanları birlikte işleme alır.
# agg({"TotalPrice":"sum"}) Total Price'ların sum'ını alır
df.groupby("Invoice").agg({"TotalPrice":"sum"}).head()

,TotalPrice
Invoice,
489434,505.30000
489435,145.80000
489436,630.33000
489437,310.75000
489438,2286.24000


- Bu tabloda Invoice'lar tekil durumda.
- Her bir faturanın toplam bedeli ne ise hesaplandı.

### Data Preparation(Veriyi Hazırlama)

In [12]:
df.shape

(525461, 9)

In [13]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
TotalPrice          0
dtype: int64

- Burdaki eksik değerleri(özellikle Customer ID) veri setinden silmemiz gerekiyor çünkü projenin amacı müşterileri segmentlere ayırmak Customer ID değeri eksik ise segmente edemeyiz.

In [14]:
# dropna fonksiyonu veri setindeki eksik değerleri siler, inplace=True ise herhangi bir atama işlemi yapmadan değişikliğin kalıcı olmasını sağlar
df.dropna(inplace=True)

In [15]:
df.shape

(417534, 9)

- Invoice değerinde başında C olan değerler iadeleri temsil etmektedir. Bu ,verinin istatistiğini çıkardığımızda eksi değerlerin gelmesine sebep  oluyor bu da anormallik olduğunu gösterir. C ile başlayan Invoice değerlerini veri setinden çıkarmamız gerekebilir.

In [16]:
# "~" dışında kalanları getir demektir.

df[~df["Invoice"].str.contains("C", na=False)] #Invoice değerinde C olanlar dışında kalanları yazdırır

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom,83.40000
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom,100.80000
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom,30.00000
...,...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95000,17530.00000,United Kingdom,5.90000
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75000,17530.00000,United Kingdom,3.75000
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75000,17530.00000,United Kingdom,3.75000
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75000,17530.00000,United Kingdom,7.50000


In [17]:
# Bunların TotalPrice'ları negatif gelir.

df[df["Invoice"].str.contains("C", na=False)] #Invoice değerinde C olanları yazdırır

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95000,16321.00000,Australia,-35.40000
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65000,16321.00000,Australia,-9.90000
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25000,16321.00000,Australia,-17.00000
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10000,16321.00000,Australia,-12.60000
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95000,16321.00000,Australia,-35.40000
...,...,...,...,...,...,...,...,...,...
524695,C538123,22956,36 FOIL HEART CAKE CASES,-2,2010-12-09 15:41:00,2.10000,12605.00000,Germany,-4.20000
524696,C538124,M,Manual,-4,2010-12-09 15:43:00,0.50000,15329.00000,United Kingdom,-2.00000
524697,C538124,22699,ROSES REGENCY TEACUP AND SAUCER,-1,2010-12-09 15:43:00,2.95000,15329.00000,United Kingdom,-2.95000
524698,C538124,22423,REGENCY CAKESTAND 3 TIER,-1,2010-12-09 15:43:00,12.75000,15329.00000,United Kingdom,-12.75000


In [18]:
# kalıcı olması için bunu dataframe'e atıyoruz

df = df[~df["Invoice"].str.contains("C", na=False)]

### Calculating RFM Metrics(RFM Metriklerinin Hesaplanması)

- Recency değerinin matematiksel karşılığı: Analizin yapıldığı tarih eksi ilgili müşterinin son satın alma işlemi yaptığı tarih.

- Burada analizi yaptığımız günü belirlemiş olmamız önemlidir.

- Bu veri seti 2009-2010 yıllarına ait olduğu ve analizi o dönemde yapmadığımız için analizin yapıldığı günü tanımlamamız gerekmektedir.

In [19]:
# veri setindeki son tarih 
df["InvoiceDate"].max() 

Timestamp('2010-12-09 20:01:00')

In [20]:
# Analizi veri setindeki son tarihten iki gün sonra yapıyoruz diyelim,11.12.2010 tarihini tarih formatında tutması için dt.datetime'ı kullanıyoruz

today_date = dt.datetime(2010,12,11) #tarih formatında tutacak bu değerleri

In [21]:
type(today_date) #zaman değişkeni olduğunu kanıtlıyor

datetime.datetime

In [22]:
rfm = df.groupby("Customer ID").agg({'InvoiceDate': lambda InvoiceDate:(today_date - InvoiceDate.max()).days,
                                     'Invoice': lambda Invoice: Invoice.nunique(), 
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

# 'InvoiceDate': lambda InvoiceDate:(today_date - InvoiceDate.max()).days bugünün tarihinden ilgili müşterinin son satın alma tarihini çıkar 
# ve bunu gün cinsinden ifade et(days ile yapar bunu) anlamına geliyor. R(recency) değeridir bu işlemin sonucu

# 'Invoice': lambda Invoice: Invoice.nunique fatura sayısını(eşsiz fatura sayısı) hesaplar bu da F(frequency) değeridir.

# 'TotalPrice': lambda TotalPrice: TotalPrice.sum() bu satır da her bir müşteri için bıraktıkları(şirkete kazandırdıkları) toplam ücrete 
# karşılık gelir yani M(monetary) değeridir.

In [23]:
rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.00000,165,11,372.86000
12347.00000,3,2,1323.32000
12348.00000,74,1,222.16000
12349.00000,43,3,2671.14000
12351.00000,11,1,300.93000


- Bu işlemler sonucunda(groupby'a alarak) Customer ID tekil durumu gelmiş yani her bir müşteri için bir satır var ve toplam değerler elde edilmiş.

In [24]:
rfm.columns = ['Recency', 'Frequency', 'Monetary']

In [25]:
rfm

,Recency,Frequency,Monetary
Customer ID,,,
12346.00000,165,11,372.86000
12347.00000,3,2,1323.32000
12348.00000,74,1,222.16000
12349.00000,43,3,2671.14000
12351.00000,11,1,300.93000
...,...,...,...
18283.00000,18,6,641.77000
18284.00000,67,1,461.68000
18285.00000,296,1,427.00000


In [26]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4314.00000,91.26982,96.94430,1.00000,18.00000,53.00000,136.00000,374.00000
Frequency,4314.00000,4.45410,8.16866,1.00000,1.00000,2.00000,5.00000,205.00000
Monetary,4314.00000,2047.28866,8912.52324,0.00000,307.95000,705.55000,1722.80250,349164.35000


- Monetary değerinde minimum değer 0 gelmiş, bu isteyeceğimiz bir durum değil. Bu nedenle bunu veri setinden çıkarmalıyız.

In [27]:
# bu işlem ile Monetary'nin 0'dan büyük olan değerlerini alırız yalnızca ve dataframe'e atarız

rfm = rfm[rfm["Monetary"]> 0] 

In [28]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4312.00000,91.17254,96.86146,1.00000,18.00000,53.00000,136.00000,374.00000
Frequency,4312.00000,4.45571,8.17021,1.00000,1.00000,2.00000,5.00000,205.00000
Monetary,4312.00000,2048.23824,8914.48128,2.95000,307.98750,706.02000,1723.14250,349164.35000


### Calculating RFM Scores(RFM Skorlarının Hesaplanması)

In [29]:
# qcut fonksiyonu quartile'lara göre bölme/ayırma işlemi yapar, önce bölmek istediğimiz değişkenin ismi verilir sonra bu değişkeni kaç parçaya
# bölmek istediğimizi yazarız ve bölme işlemi sonrası kullanılacak(atanacak) label'ları(etiketleri) yazarız
# qcut fonksiyonu bir değişkeni küçükten büyüğe sıralar ve belirli parçalara göre bunu böler segmente eder de diyebiliriz 

rfm["recency_score"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
# recency değerinin küçük olması bizim için daha iyiydi bu nedenle küçük recency değerlerine 5 etiketi atanır, atamalar bu şekilde olur


# 0-100 arasındaki değerleri 5 parçaya bölelim; 0-20, 20-40, 40-60, 60-80, 80-100

C:\Users\Monster\AppData\Local\Temp\ipykernel_12160\981371892.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["recency_score"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])


In [30]:
rfm.head()

,Recency,Frequency,Monetary,recency_score
Customer ID,,,,
12346.00000,165,11,372.86000,2
12347.00000,3,2,1323.32000,5
12348.00000,74,1,222.16000,2
12349.00000,43,3,2671.14000,3
12351.00000,11,1,300.93000,5


In [31]:
# frequency değerlerinde tekrar eden çok fazla  değer old. için sağlıklı bir sıralama ve bölme işlemi olmuyor bu nedenle rank metodu ile
# sıraladıktan sonra ilk gördüğünü ilk değere ata diyoruz ve sorun halloluyor
rfm["frequency_score"] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

rfm["monetary_score"] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

C:\Users\Monster\AppData\Local\Temp\ipykernel_12160\3928402141.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["frequency_score"] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
C:\Users\Monster\AppData\Local\Temp\ipykernel_12160\3928402141.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["monetary_score"] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])


In [32]:
rfm

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score
Customer ID,,,,,,
12346.00000,165,11,372.86000,2,5,2
12347.00000,3,2,1323.32000,5,2,4
12348.00000,74,1,222.16000,2,1,1
12349.00000,43,3,2671.14000,3,3,5
12351.00000,11,1,300.93000,5,1,2
...,...,...,...,...,...,...
18283.00000,18,6,641.77000,4,5,3
18284.00000,67,1,461.68000,3,2,2
18285.00000,296,1,427.00000,1,2,2


In [33]:
# R ve F skorlarını kullanıyoruz yalnızca 
# astype(str) ile skorları string'e çevirdik
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str))

rfm

C:\Users\Monster\AppData\Local\Temp\ipykernel_12160\60634636.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +


,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12346.00000,165,11,372.86000,2,5,2,25
12347.00000,3,2,1323.32000,5,2,4,52
12348.00000,74,1,222.16000,2,1,1,21
12349.00000,43,3,2671.14000,3,3,5,33
12351.00000,11,1,300.93000,5,1,2,51
...,...,...,...,...,...,...,...
18283.00000,18,6,641.77000,4,5,3,45
18284.00000,67,1,461.68000,3,2,2,32
18285.00000,296,1,427.00000,1,2,2,12


In [34]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4312.00000,91.17254,96.86146,1.00000,18.00000,53.00000,136.00000,374.00000
Frequency,4312.00000,4.45571,8.17021,1.00000,1.00000,2.00000,5.00000,205.00000
Monetary,4312.00000,2048.23824,8914.48128,2.95000,307.98750,706.02000,1723.14250,349164.35000


In [35]:
# RFM_SCORE değeri de string formatındadır
# Champions(şampiyonlar) grubu kimmiş bakalım
# rfm skoru 55 olanları yazdıracak
rfm[rfm["RFM_SCORE"] == "55"]

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12415.00000,11,7,19543.84000,5,5,5,55
12431.00000,9,13,4370.52000,5,5,5,55
12471.00000,10,49,20139.74000,5,5,5,55
12472.00000,5,13,11308.48000,5,5,5,55
12474.00000,14,13,5048.66000,5,5,5,55
...,...,...,...,...,...,...,...
18225.00000,1,15,7545.14000,5,5,5,55
18226.00000,14,15,6650.83000,5,5,5,55
18229.00000,2,10,3526.81000,5,5,5,55


In [36]:
rfm[rfm["RFM_SCORE"] == "11"]

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12355.00000,203,1,488.21000,1,1,2,11
12362.00000,374,1,130.00000,1,1,1,11
12366.00000,269,1,500.24000,1,1,2,11
12368.00000,264,1,917.70000,1,1,3,11
12378.00000,198,1,1407.70000,1,1,4,11
...,...,...,...,...,...,...,...
15928.00000,292,1,293.53000,1,1,2,11
15929.00000,280,1,594.00000,1,1,3,11
15941.00000,273,1,405.00000,1,1,2,11


### Creating & Analysing RFM Segments(RFM Segmentlerinin Oluşturulması ve Analiz Edilmesi)

In [37]:
# regex(regular expressions)

# RFM isimlendirmesi
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

# r'[1-2][1-2]': 1. elemanında 1 ya da 2, 2.elemanında da 1 ya da 2 görürsen "hibernating" etiketini yapıştır demektir
# r'41' : RFM skorunun 1. elemanında 4, 2. elemanında 1 görürsen promising etiketini yapıştır demektir

# 'RFM_SCORE' değerlerini 'seg_map' ile değiştirir
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True) 

C:\Users\Monster\AppData\Local\Temp\ipykernel_12160\3850992590.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)


In [38]:
rfm

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.00000,165,11,372.86000,2,5,2,25,cant_loose
12347.00000,3,2,1323.32000,5,2,4,52,potential_loyalists
12348.00000,74,1,222.16000,2,1,1,21,hibernating
12349.00000,43,3,2671.14000,3,3,5,33,need_attention
12351.00000,11,1,300.93000,5,1,2,51,new_customers
...,...,...,...,...,...,...,...,...
18283.00000,18,6,641.77000,4,5,3,45,loyal_customers
18284.00000,67,1,461.68000,3,2,2,32,about_to_sleep
18285.00000,296,1,427.00000,1,2,2,12,hibernating


In [39]:
# segment sütununa(özelliğine göre) groupby'a alıp  her bir grupta kaç kişi var ortalamaları neler hesaplanıyor
rfm[["segment", "Recency", "Frequency", "Monetary"]].groupby("segment").agg(["mean", "count"])

Recency       Frequency         Monetary      
                         mean count      mean count       mean count
segment                                                             
about_to_sleep       53.81924   343   1.20117   343  441.32000   343
at_Risk             152.15876   611   3.07365   611 1188.87832   611
cant_loose          124.11688    77   9.11688    77 4099.45000    77
champions             7.11916   663  12.55354   663 6852.26417   663
hibernating         213.88571  1015   1.12611  1015  403.97784  1015
loyal_customers      36.28706   742   6.83019   742 2746.06735   742
need_attention       53.26570   207   2.44928   207 1060.35700   207
new_customers         8.58000    50   1.00000    50  386.19920    50
potential_loyalists  18.79304   517   2.01741   517  729.51099   517
promising            25.74713    87   1.00000    87  367.08678    87

In [40]:
# oluşan segmentlerden "cant_loose" sınıfına dahil olanları yazdırır ekrana 
rfm[rfm["segment"] == "cant_loose"].head()

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.00000,165,11,372.86000,2,5,2,25,cant_loose
12380.00000,101,7,6951.49000,2,5,5,25,cant_loose
12482.00000,212,29,23691.40000,1,5,5,15,cant_loose
12510.00000,95,7,4195.45000,2,5,5,25,cant_loose
12891.00000,94,8,509.50000,2,5,3,25,cant_loose


In [41]:
# oluşan segmentlerden "cant_loose" sınıfına dahil olanların ID'lerine ulaşmak istersek
rfm[rfm["segment"] == "cant_loose"].index

Index([12346.0, 12380.0, 12482.0, 12510.0, 12891.0, 12932.0, 13044.0, 13313.0,
       13680.0, 13782.0, 13799.0, 13856.0, 14025.0, 14063.0, 14160.0, 14221.0,
       14548.0, 14607.0, 14685.0, 14745.0, 15003.0, 15013.0, 15015.0, 15125.0,
       15141.0, 15222.0, 15306.0, 15321.0, 15359.0, 15369.0, 15372.0, 15443.0,
       15538.0, 15607.0, 15633.0, 15722.0, 15751.0, 15754.0, 15768.0, 15911.0,
       15912.0, 16027.0, 16032.0, 16158.0, 16177.0, 16197.0, 16335.0, 16467.0,
       16631.0, 16742.0, 16743.0, 16875.0, 16986.0, 17021.0, 17032.0, 17092.0,
       17113.0, 17157.0, 17188.0, 17230.0, 17268.0, 17426.0, 17448.0, 17454.0,
       17512.0, 17578.0, 17602.0, 17651.0, 17940.0, 17969.0, 17988.0, 18009.0,
       18051.0, 18064.0, 18094.0, 18251.0, 18258.0],
      dtype='float64', name='Customer ID')

In [42]:
new_df = pd.DataFrame()
new_df["new_customer_id"] = rfm[rfm["segment"] == "new_customers"].index

new_df["new_customer_id"] = new_df["new_customer_id"].astype(int)

new_df.to_csv("new_customers.csv")

In [43]:
rfm.to_csv("rfm.csv") # oluşturduğumuz rfm tablosunun tamamını bir csv dosyasına yazar

### Functionalization(Tüm Sürecin Fonksiyonlaştırılması)

In [44]:
# daha önceden yapılmış olan işlemler tek bir fonksiyon altında birleştiriliyor

def create_rfm(dataframe, csv=False): #default olarak csv oluşturma demektir "csv=False"

    # VERIYI HAZIRLAMA
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"] #total price hesaplanıyor
    dataframe.dropna(inplace=True) # eksik değerler veri setinden atılıyor
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)] # Invoice'u C ile başlayanlar atılmış 

    # RFM METRIKLERININ HESAPLANMASI
    today_date = dt.datetime(2011, 12, 11)
    rfm = dataframe.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                                'Invoice': lambda num: num.nunique(),
                                                "TotalPrice": lambda price: price.sum()})
    rfm.columns = ['Recency', 'Frequency', "Monetary"]
    rfm = rfm[(rfm['Monetary'] > 0)]

    # RFM SKORLARININ HESAPLANMASI
    rfm["recency_score"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
    rfm["frequency_score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
    rfm["monetary_score"] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

    # cltv_df skorları kategorik değere dönüştürülüp df'e eklendi
    rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                        rfm['frequency_score'].astype(str))


    # SEGMENTLERIN ISIMLENDIRILMESI
    seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }

    rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)
    rfm = rfm[["Recency", "Frequency", "Monetary", "segment"]]
    rfm.index = rfm.index.astype(int) #customer ID'leri integer olarak alırız

    # eğer csv argümanına bir değer girildiyse yani True yazıyorsa aşağıdaki işlemi yap
    if csv:
        rfm.to_csv("rfm.csv") # rfm'in csv dosyasını oluştur

    return rfm

In [45]:
df = df_.copy() # dataframe'i ilk haline, veri setinden okunan ilk haline geri getiriyoruz

rfm_new = create_rfm(df, csv=True) #csv dosyasına yazmak için csv=True özelliğini aktive ediyoruz